In [1]:
from utils import read_csv_data, clean_location, build_column_vocabulary

In [2]:
data = read_csv_data("../data/processed_job_postings.csv", ["name", "industry", "location", "formatted_experience_level"], "standardized_annual_salary")
data = clean_location(data, 2)
import random
random.seed(42)
random.shuffle(data)

train_data = data[:10000]
val_data = data[10000:11000]
test_data = data[12000:]

In [3]:
print(train_data[10])



(['Hallmark Cards', 'Retail', 'FL', 'Entry level'], '26000.0')


In [4]:
vocab = build_column_vocabulary(train_data, 2)
print(len(vocab))
print(vocab.get_stoi())

56
{'SD': 54, 'VT': 52, 'QC': 55, 'AK': 51, 'HI': 49, 'MT': 48, 'MS': 47, 'WY': 53, 'ID': 43, 'DE': 40, 'AR': 42, 'NE': 38, 'NV': 31, 'AL': 35, 'SC': 27, 'KS': 37, 'VA': 11, 'OK': 34, 'IN': 25, 'NM': 41, 'UT': 29, 'MN': 21, 'DC': 33, 'AZ': 17, 'WA': 7, 'NJ': 13, 'MI': 20, 'RI': 46, 'ME': 44, 'OH': 18, '<eos>': 1, 'MA': 14, '<pad>': 3, 'IL': 10, 'NH': 36, 'PA': 15, '<unk>': 2, 'CO': 9, 'OR': 24, 'MO': 26, 'FL': 8, 'LA': 39, 'CT': 28, 'GA': 12, 'TX': 6, 'ND': 45, 'WI': 23, 'NY': 5, 'IA': 32, 'CA': 4, 'KY': 30, 'MD': 19, '<bos>': 0, 'WV': 50, 'TN': 22, 'NC': 16}
